# Understanding NHS digital vaccination certificates

In [1]:
from pyzbar.pyzbar import decode as qrdecode
from PIL import Image
import zlib, base45, base64, cbor2

The NHS app offers two types of certificates:
 * 'Coronavirus (COVID-19) vaccination records'
 * [NHS COVID Pass](https://www.gov.uk/guidance/demonstrating-your-covid-19-status)
 
 
 
## Coronavirus (COVID-19) vaccination records

These appear to be [EU standard conforming](https://ec.europa.eu/health/ehealth/covid-19_en). But since isn't taking part in the EU Digital greent certificate initiative, these will not be validated by cert reading apps of countries. Malta was complaining about this. The ground work has been done, the data is compatible, the [EU is happy to have other countries take part too](https://ec.europa.eu/info/live-work-travel-eu/coronavirus-response/safe-covid-19-vaccines-europeans/eu-digital-covid-certificate_en) - it all just seems political.

There is a 'blueprint' implementation of various useful apps and specifications on [the EU's github organisation](https://github.com/eu-digital-green-certificates). It should really be straight forward to add Englands public keys to [that app](https://github.com/eu-digital-green-certificates/dgca-verifier-app-android) so that they can also be verified.




In [2]:
## from https://covid-status.service.nhsx.nhs.uk/pubkeys/keys.json
keys = [{
	"kid": "S2V5MVJF",
	"publicKey": "MFkwEwYHKoZIzj0CAQYIKoZIzj0DAQcDQgAEtWokvmqrJOv/0PO9Vy8lpb6SgWw+rao0qIXntO/Bf7ExryL3yyKRI73IqAh38Lk4joqHrZK8XLZV9PMclgmTVg=="
}, {
	"kid": "S2V5MlJF",
	"publicKey": "MFkwEwYHKoZIzj0CAQYIKoZIzj0DAQcDQgAEAauvVllnjXm0toiI2cUQfCTdZiDQ6kvtoo1bSnl8W3Rq7WyOeHWYKhBaQ4rEBceqNl5+v1ZLGj0WfnIhXh246Q=="
}, {
	"kid": "S2V5M1JF",
	"publicKey": "MFkwEwYHKoZIzj0CAQYIKoZIzj0DAQcDQgAE2ZpDNa1VV6g2PkmyKoL1INO0MtTqE5WT45i3QhY9FFMjbF9ieqnHV4R814wrN3f3vzkx0VN/YJZH4rI1GDlfqw=="
}, {
	"kid": "S2V5NFJF",
	"publicKey": "MFkwEwYHKoZIzj0CAQYIKoZIzj0DAQcDQgAE8g5iFLRT7NyRmKp7pcP8uEgoHFhfmcXOLLD4RUtX50/Rh4Cz7l/faAiODNMmCkcWLA1Z8WOZoNFQsdmeDjXuLQ=="
}, {
	"kid": "S2V5NVJF",
	"publicKey": "MFkwEwYHKoZIzj0CAQYIKoZIzj0DAQcDQgAEOFOUp1+SLuaM3NnV+OMZKZOoPg76T7D+vqRCasD0BrRZmlUH2gD+aVlpKvp+u7h8ywTR7T6Z6/iI2Qe6F5begg=="
}, {
	"kid": "S2V5MVBSTw==",
	"publicKey": "MFkwEwYHKoZIzj0CAQYIKoZIzj0DAQcDQgAEyfUqlGg4hfyPYgZJzl1KepkfTaX+F3592syCJ0ylVOHA4fE6vavLXE4cG+Whz/eqyW/rFuZ0HBHHEskmpCngOA=="
}, {
	"kid": "S2V5MlBSTw==",
	"publicKey": "MFkwEwYHKoZIzj0CAQYIKoZIzj0DAQcDQgAEP6P6ZlbhcW1xZpp91qagUY+iLIyuu+CynzAlrqiiseqmOYH8uJ71CkbEYhbVh8TemnbaR0unE2j9EPK7Y/x7Vg=="
}, {
	"kid": "S2V5M1BSTw==",
	"publicKey": "MFkwEwYHKoZIzj0CAQYIKoZIzj0DAQcDQgAErltiOh3nmU+x4p5r249O/2fBSnHkjJpas23lhMAtEYeQutHiw0G+zeEUNZ/n++/XbFoY5hH68d27cGaGW6uoxw=="
}, {
	"kid": "S2V5NFBSTw==",
	"publicKey": "MFkwEwYHKoZIzj0CAQYIKoZIzj0DAQcDQgAEEfIefhjjacwZ+xxKudJGqdHr0j95tWtYIUGPeWV2XWmduygfe3oIFJu/A2kYGmeZ4u/ERSTNM2ZXuE9k19xVJQ=="
}, {
	"kid": "S2V5NVBSTw==",
	"publicKey": "MFkwEwYHKoZIzj0CAQYIKoZIzj0DAQcDQgAE+4ukUdmm9f4AmyQBwTLQFKPPCQj4fP2BpQkIl2hb4p6FxPGDP9z3JkIo2w2xkoxqs2JrTHn2MCdyh3aeyZWD4A=="
}]

In [3]:
## my personal vaccination certificate, not part of this repo - contains PII
nhsDataOrig = qrdecode(Image.open('nhs-vaccination-certificate.png'))[0]

In [4]:
nhsDataOrig.data[:4]

b'HC1:'

In [5]:
len(nhsDataOrig.data[4:])

518

In [6]:
## so its got the right starting string

In [7]:
nhsData = cbor2.loads(zlib.decompress(base45.b45decode(nhsDataOrig.data[4:])))

In [8]:
from copy import deepcopy
def sanitise(d):
    d = deepcopy(d)
    d[4] = 1620000000
    d[6] = 1630000000
    d[-260][1]['v'][0]['ci'] = 'URN:UVCI:01:GB:...'
    d[-260][1]['v'][0]['dt'] = '2021-06-01'
    d[-260][1]['dob'] = '1900-01-01'
    d[-260][1]['nam'] = {'fn' : 'SURNAME', 'gn' : 'Firstname', 'fnt' : 'SURNAME', 'gnt': 'FIRSTNAME'}
    return d

In [9]:
## All the details! Seem entirely standard conforming.
sanitise(cbor2.loads(nhsData.value[2]))

{1: 'GB',
 4: 1620000000,
 6: 1630000000,
 -260: {1: {'v': [{'ci': 'URN:UVCI:01:GB:...',
     'co': 'GB',
     'dn': 2,
     'dt': '2021-06-01',
     'is': 'NHS Digital',
     'ma': 'ORG-100001699',
     'mp': 'EU/1/21/1529',
     'sd': 2,
     'tg': '840539006',
     'vp': '1119305005',
     'lot': 'PW40040'}],
   'dob': '1900-01-01',
   'nam': {'fn': 'SURNAME',
    'gn': 'Firstname',
    'fnt': 'SURNAME',
    'gnt': 'FIRSTNAME'},
   'ver': '1.0.0'}}}

In [10]:
# This is the key identifier
cbor2.loads(nhsData.value[0])

{1: -7, 4: b'Key5PRO'}

In [11]:
pk = next(x['publicKey'] for x in keys if base64.b64decode(x['kid']) == cbor2.loads(nhsData.value[0])[4])
## ok, so the PK is 
pk

'MFkwEwYHKoZIzj0CAQYIKoZIzj0DAQcDQgAE+4ukUdmm9f4AmyQBwTLQFKPPCQj4fP2BpQkIl2hb4p6FxPGDP9z3JkIo2w2xkoxqs2JrTHn2MCdyh3aeyZWD4A=='

Cose needs the $x$ and $y$ (or $r$ and $s$) factors of the elliptic curve to validate the signature. I am not sure how to get these though currently. [8gwifi] has a pem parser function that gives them to me - select `Public Key` on that website, and stick 

```
-----BEGIN PUBLIC KEY-----
MFkwEwYHKoZIzj0CAQYIKoZIzj0DAQcDQgAE+4ukUdmm9f4AmyQBwTLQFKPPCQj4fP2BpQkIl2hb4p6FxPGDP9z3JkIo2w2xkoxqs2JrTHn2MCdyh3aeyZWD4A==
-----END PUBLIC KEY-----
```

But we can also get these numbers in python.

In [12]:
from pyasn1.codec.der.decoder import decode

In [13]:
received_record, rest_of_substrate = decode(base64.b64decode(pk))

In [14]:
for field in received_record:
    print('{} is {}'.format(field, received_record[field]))

field-0 is SequenceOf:
 1.2.840.10045.2.1 1.2.840.10045.3.1.7
field-1 is 0000010011111011100010111010010001010001110110011010011011110101111111100000000010011011001001000000000111000001001100101101000000010100101000111100111100001001000010001111100001111100111111011000000110100101000010010000100010010111011010000101101111100010100111101000010111000100111100011000001100111111110111001111011100100110010000100010100011011011000011011011000110010010100011000110101010110011011000100110101101001100011110011111011000110000001001110111001010000111011101101001111011001001100101011000001111100000


In [15]:
pkBin = received_record[1].asOctets()
pkBin

b"\x04\xfb\x8b\xa4Q\xd9\xa6\xf5\xfe\x00\x9b$\x01\xc12\xd0\x14\xa3\xcf\t\x08\xf8|\xfd\x81\xa5\t\x08\x97h[\xe2\x9e\x85\xc4\xf1\x83?\xdc\xf7&B(\xdb\r\xb1\x92\x8cj\xb3bkLy\xf60'r\x87v\x9e\xc9\x95\x83\xe0"

so starting with 04 - uncompressed two numbers, but the encoding seems difficult to get?
 * https://stackoverflow.com/questions/16899247/how-can-i-decode-a-ssl-certificate-using-python/50072461 
 * https://superuser.com/questions/900918/get-x-and-y-components-of-ec-public-key-using-openssl
 
 

In [16]:
len(pkBin)

65

In [17]:
X = pkBin[1:33]
Y = pkBin[33:]

In [18]:
## lets use that to validate the certificate.

In [19]:
from cose.messages import CoseMessage
from cose.keys import CoseKey
from cose.algorithms import EdDSA
from cose.keys.curves import P256
from cose.keys.keyparam import KpKty, KpKeyOps, EC2KpCurve, EC2KpX, EC2KpY
from cose.keys.keytype import KtyEC2
from cose.keys.keyops import VerifyOp

In [20]:
cose_key = {
    KpKty: KtyEC2,
    EC2KpCurve: P256,
    KpKeyOps: [VerifyOp],
    EC2KpX: X,
    EC2KpY: Y,
}

cose_key = CoseKey.from_dict(cose_key)

In [21]:
# decode and verify the signature

decoded = CoseMessage.decode(zlib.decompress(base45.b45decode(nhsDataOrig.data[4:])))

decoded

<COSE_Sign1: [{'Algorithm': 'Es256', 'KID': b'Key5PRO'}, {}, b'\xa4\x01bGB' ... (257 B), b'M\xf0\x17[\xa3' ... (64 B)]>

In [22]:
decoded.key = cose_key

decoded.verify_signature()

True

Peachy!

# NHS COVID Pass

@martinbrook released his [Covid Pass certificate on github](https://github.com/eu-digital-green-certificates/dcc-quality-assurance/issues/64). I will be taking a look at that one, but again, the qr code is not contained in this repo. Download it from the github issue, take a png screenshot of the qr code, and save it as `covid-pass-certificate.png` for the code below to work.

The [NHS COVID Pass](https://www.gov.uk/guidance/demonstrating-your-covid-19-status) seems to be what the government wants to use for international travel - rather than the vaccination certificates which should contain all relevant information already? I am confused.

There are some [guidance on how to use this](https://www.nhsx.nhs.uk/covid-19-response/using-the-nhs-covid-pass/), including a ['verifier app'](https://www.nhsx.nhs.uk/covid-19-response/international-covid-pass-verifier-app-user-guide/). However I am unable to find this app in the [google play store](https://play.google.com/store/search?q=NHS%20COVID%20Pass%20Verifier%20app), and a [telegraph article confirms that the app has yet to be released](https://www.telegraph.co.uk/news/2021/07/15/covid-passport-backlash-pubs-restaurants-say-cant-check-qr-codes/).

In [27]:
covidPassOrig = qrdecode(Image.open('covid-pass-certificate.png'))[0]

In [24]:
len(covidPassOrig.data)

131

In [25]:
covidPassOrig.data[:30]

b'S2V5MlBSTw==.MTIxMDYzMDE1NDJNQ'

This has a different data structure to the Vaccination records. 

 * The initial part seems to match one of the keys from the nhs key list
 * Otherwise the NHS has not released any information on the encoding. It seems to contain quite a bit less data than the vaccination certificates (131 bytes vs 518 bytes in my case)
     

In [26]:
pkCP = next(x['publicKey'] for x in keys if x['kid'] == covidPassOrig.data[:12].decode('ascii'))
## ok, so the PK is 
pkCP

'MFkwEwYHKoZIzj0CAQYIKoZIzj0DAQcDQgAEP6P6ZlbhcW1xZpp91qagUY+iLIyuu+CynzAlrqiiseqmOYH8uJ71CkbEYhbVh8TemnbaR0unE2j9EPK7Y/x7Vg=='

Can't really do much more here. 